# Protocol 1: Bond-Type Rotation Experiment
## Non-Abelian Stratified Quantum Normative Dynamics (NA-SQND)

This notebook tests the prediction that crossing moral thresholds induces systematic rotation in bond-type space.

**Supports:**
- 🦙 Ollama (local models like Llama, Mistral, etc.)
- 🤖 Anthropic Claude API
- 🧪 Simulation mode (no API needed)

**Theoretical Prediction:**
$$|b_{\text{final}}\rangle = U(\gamma)|b_{\text{init}}\rangle = \cos(\theta/2)|O\rangle + i\sin(\theta/2)|C\rangle$$

Where θ increases with threshold strength: Control (0°) → Weak (30°) → Strong (60°)

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install -q anthropic requests pandas numpy scipy matplotlib seaborn

In [ ]:
# For Colab: Install and run Ollama (optional - only if using local models)
# Uncomment the following lines if you want to use Ollama on Colab:

# !curl -fsSL https://ollama.com/install.sh | sh
# !nohup ollama serve > ollama.log 2>&1 &
# import time; time.sleep(5)  # Wait for server to start
# !ollama pull llama3.2  # or mistral, phi3, etc.

In [ ]:
import random
import time
import requests
from datetime import datetime
from dataclasses import dataclass, field
from typing import List, Dict, Tuple
from enum import Enum
from abc import ABC, abstractmethod

import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

print("✓ All imports successful!")

## 2. Configuration

Choose your backend and set parameters:

In [ ]:
#@title Configuration { display-mode: "form" }

#@markdown ### Backend Selection
BACKEND = "simulation"  #@param ["simulation", "ollama", "anthropic"]

#@markdown ### Ollama Settings (if using Ollama)
OLLAMA_MODEL = "llama3.2"  #@param ["llama3.2", "llama3.1", "mistral", "phi3", "gemma2"]
OLLAMA_URL = "http://localhost:11434"  #@param {type:"string"}

#@markdown ### Anthropic Settings (if using Claude)
ANTHROPIC_API_KEY = ""  #@param {type:"string"}
ANTHROPIC_MODEL = "claude-sonnet-4-20250514"  #@param ["claude-sonnet-4-20250514", "claude-haiku-4-20250514"]

#@markdown ### Experiment Settings
N_SCENARIOS = 30  #@param {type:"slider", min:5, max:150, step:5}
CONDITIONS = ["control", "weak", "strong"]  # Fixed

print(f"Backend: {BACKEND}")
print(f"Scenarios: {N_SCENARIOS}")
print(f"Total evaluations: {N_SCENARIOS * len(CONDITIONS)}")

## 3. Core Data Structures

In [ ]:
class BondType(Enum):
    """Hohfeldian incident relations."""
    OBLIGATION = "O"  # Duty
    CLAIM = "C"       # Right
    LIBERTY = "L"     # Permission (no duty)
    NO_CLAIM = "N"    # No right
    
    @classmethod
    def from_string(cls, s: str) -> 'BondType':
        """Parse bond type from string."""
        s = s.strip().upper()
        if s in ['O', 'OBLIGATION', 'DUTY']:
            return cls.OBLIGATION
        elif s in ['C', 'CLAIM', 'RIGHT']:
            return cls.CLAIM
        elif s in ['L', 'LIBERTY', 'PERMISSION', 'FREE']:
            return cls.LIBERTY
        elif s in ['N', 'NO_CLAIM', 'NO-CLAIM', 'NO CLAIM', 'NOCLAIM']:
            return cls.NO_CLAIM
        else:
            raise ValueError(f"Unknown bond type: {s}")


@dataclass
class AITAScenario:
    """Represents an AITA scenario."""
    id: str
    title: str
    text: str
    label: str
    agent_a: str = "the poster"
    agent_b: str = "the other party"


@dataclass
class ThresholdCondition:
    """Represents a threshold manipulation."""
    name: str
    description: str
    prompt_modifier: str


@dataclass
class EvaluationResult:
    """Result from a single evaluation."""
    scenario_id: str
    condition: str
    bond_type: BondType
    confidence: float
    reasoning: str
    raw_response: str
    model: str = ""
    timestamp: datetime = field(default_factory=datetime.now)


# Define threshold conditions
THRESHOLD_CONDITIONS = {
    "control": ThresholdCondition(
        name="control",
        description="No additional context (baseline evaluation)",
        prompt_modifier=""
    ),
    "weak": ThresholdCondition(
        name="weak",
        description="Minor context shift",
        prompt_modifier="""\nAdditionally, consider that the other party has explained their perspective 
and their actions were based on a misunderstanding. They did not intend any harm 
and have expressed willingness to discuss the matter.\n"""
    ),
    "strong": ThresholdCondition(
        name="strong",
        description="Major context shift",
        prompt_modifier="""\nIMPORTANT UPDATE: New information has come to light that fundamentally changes 
the situation. The other party was acting under significant personal distress 
(a family emergency) that the poster was unaware of. The other party has since 
apologized and explained the circumstances. The immediate conflict has been resolved.\n"""
    )
}

print("✓ Data structures defined!")

## 4. Sample AITA Dataset

In [ ]:
def get_sample_aita_dataset() -> List[AITAScenario]:
    """Returns a sample AITA dataset."""
    return [
        AITAScenario(
            id="aita_001",
            title="AITA for not attending my sister's wedding?",
            text="""My sister is getting married next month. She scheduled it on the same weekend 
as my PhD dissertation defense, which I scheduled 6 months ago and cannot change. I told her 
about the conflict when she first announced the date, but she said I should reschedule my 
defense. My parents are furious with me for 'choosing my career over family.'""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_002",
            title="AITA for refusing to lend money to my brother?",
            text="""My brother asked me to lend him $5000 for a 'business opportunity.' He has 
a history of failed ventures and still owes me $2000 from three years ago. When I said no, 
he called me selfish and said family should help family.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_003",
            title="AITA for telling my roommate to stop having loud parties?",
            text="""My roommate has loud parties every weekend until 3-4 AM. I work early shifts 
and need sleep. I've asked nicely multiple times but nothing changes. Last weekend I finally 
told all the guests to leave at midnight. My roommate says I embarrassed them.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_004",
            title="AITA for not sharing my inheritance with siblings?",
            text="""My grandmother left me her house (~$400k) in her will. My siblings got $50k 
each. They're upset and want me to sell and split equally. But grandma left me the house 
because I was her caretaker for 5 years while they rarely visited.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_005",
            title="AITA for reporting my coworker for time theft?",
            text="""I noticed my coworker regularly clocks in then leaves for 1-2 hours while 
staying on the clock. I mentioned it to our manager who investigated and my coworker got 
written up. Now everyone calls me a snitch.""",
            label="ESH"
        ),
        AITAScenario(
            id="aita_006",
            title="AITA for refusing to cook separate meals for my picky husband?",
            text="""My husband won't eat vegetables or anything 'ethnic.' I've been cooking 
two separate dinners every night for years. I finally told him he can make his own food 
or eat what I make. He says I'm being unreasonable.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_007",
            title="AITA for not letting my parents move in with me?",
            text="""My parents want to move in to save money for retirement. I have a 3-bedroom 
house and live alone. But they're very controlling - living with them was miserable. I offered 
to help find them affordable housing instead. They call me ungrateful.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_008",
            title="AITA for telling my friend her wedding dress is ugly?",
            text="""My best friend showed me her wedding dress and asked for my honest opinion. 
It's objectively unflattering. I gently said it might not be the most flattering choice. 
She burst into tears - she already bought it non-refundable.""",
            label="YTA"
        ),
        AITAScenario(
            id="aita_009",
            title="AITA for leaving my boyfriend at a restaurant?",
            text="""My boyfriend made a sexist joke at dinner with his friends. When I didn't 
laugh, he made another one. I said I was leaving. He laughed and said I was overreacting. 
I took an Uber home. He's furious I left him there.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_010",
            title="AITA for not going to my nephew's birthday party?",
            text="""My sister expects everyone to attend every single one of her 4 kids' events. 
I told her I can't make it to my nephew's 7th birthday because I have plans with friends 
I haven't seen in months. She's accusing me of not caring about family.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_011",
            title="AITA for refusing to give up my plane seat?",
            text="""I paid extra for an aisle seat due to a medical condition (blood clot risk). 
A mother asked me to switch with her middle seat so she could sit with her kids. I explained 
my medical need and refused. She loudly called me selfish.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_012",
            title="AITA for kicking out my adult son?",
            text="""My 25-year-old son has been living at home rent-free for 3 years. He doesn't 
work, doesn't help around the house, and plays video games all day. I gave him 3 months notice 
to find a job and move out. My wife thinks I'm being too harsh.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_013",
            title="AITA for not tipping on a takeout order?",
            text="""I ordered takeout and the tablet suggested 20-30% tip for an order I picked 
up myself. I selected 'no tip' and the cashier gave me a dirty look. My friend said I was cheap.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_014",
            title="AITA for calling police on my neighbor's dog?",
            text="""My neighbor's dog barks constantly - early morning, late night. I've asked 
them multiple times. They said dogs bark and I should deal with it. After 6 months, I filed 
a noise complaint. Now they're furious.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_015",
            title="AITA for reading my teenager's diary?",
            text="""I read my 15-year-old's diary because I was worried about their behavior. 
I found they've been vaping and lying about where they go. When I confronted them, they were 
more upset about the privacy violation than anything.""",
            label="ESH"
        ),
        AITAScenario(
            id="aita_016",
            title="AITA for not visiting my sick grandmother?",
            text="""My grandmother is in the hospital but I live 1000 miles away and can't 
afford to fly. I've been calling every day but my family says I should find a way to visit. 
I just started a new job and can't take time off.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_017",
            title="AITA for eating my roommate's food?",
            text="""I ate my roommate's leftovers from the fridge. They were unlabeled and had 
been there for 5 days. My roommate is furious - apparently they were saving it for lunch today. 
I offered to replace it but they're still mad.""",
            label="YTA"
        ),
        AITAScenario(
            id="aita_018",
            title="AITA for not helping my friend move?",
            text="""My friend asked me to help them move on a Saturday. I said no because I had 
already committed to a birthday party. They said I could skip the party since 'it's just a 
party.' They ended up hiring movers and are bitter.""",
            label="NTA"
        ),
        AITAScenario(
            id="aita_019",
            title="AITA for correcting someone's grammar?",
            text="""A coworker sent an email with several grammar mistakes to the whole team. 
I replied-all with corrections. I thought I was being helpful but they were embarrassed 
and HR talked to me about professionalism.""",
            label="YTA"
        ),
        AITAScenario(
            id="aita_020",
            title="AITA for not sharing lottery winnings?",
            text="""I won $50,000 in the lottery. My family found out and now everyone expects 
a share. I'm using it to pay off my student loans and won't have much left. They say I'm 
selfish for not sharing my 'windfall.'""",
            label="NTA"
        ),
    ]

# Load dataset
SCENARIOS = get_sample_aita_dataset()
print(f"✓ Loaded {len(SCENARIOS)} sample scenarios")

In [ ]:
# Optional: Upload your own AITA dataset
# Uncomment and run this cell to upload a CSV file

# from google.colab import files
# uploaded = files.upload()
# 
# if uploaded:
#     filename = list(uploaded.keys())[0]
#     df = pd.read_csv(filename)
#     SCENARIOS = [
#         AITAScenario(
#             id=str(row.get('post_id', f'aita_{i}')),
#             title=str(row.get('post_title', 'AITA scenario')),
#             text=str(row.get('post_content', '')),
#             label=str(row.get('verdict', 'UNKNOWN'))
#         )
#         for i, row in df.iterrows()
#     ]
#     print(f"✓ Loaded {len(SCENARIOS)} scenarios from {filename}")

## 5. Evaluator Backends

In [ ]:
class BaseEvaluator(ABC):
    """Abstract base class for evaluators."""
    
    def __init__(self, model: str):
        self.model = model
        self.request_count = 0
    
    def create_prompt(self, scenario: AITAScenario, condition: ThresholdCondition) -> str:
        """Create the evaluation prompt."""
        return f"""You are evaluating a moral scenario using the Hohfeldian framework.

SCENARIO:
{scenario.text}
{condition.prompt_modifier}
YOUR TASK:
Analyze the moral relationship between {scenario.agent_a} and {scenario.agent_b}.

Classify the PRIMARY moral relation as ONE of:
- O (OBLIGATION): {scenario.agent_a} has a duty toward {scenario.agent_b}
- C (CLAIM): {scenario.agent_b} has a right against {scenario.agent_a}
- L (LIBERTY): {scenario.agent_a} is free to act as they did (no duty)
- N (NO-CLAIM): {scenario.agent_b} has no right against {scenario.agent_a}

Respond in this EXACT format (no other text):
CLASSIFICATION: [O/C/L/N]
CONFIDENCE: [0.0-1.0]
REASONING: [1-2 sentences]
"""
    
    @abstractmethod
    def evaluate(self, scenario: AITAScenario, condition: ThresholdCondition) -> EvaluationResult:
        pass
    
    def parse_response(self, response: str, scenario_id: str, condition: str) -> EvaluationResult:
        """Parse the model's response."""
        bond_type = BondType.LIBERTY
        confidence = 0.5
        reasoning = ""
        
        for line in response.strip().split('\n'):
            line = line.strip()
            if line.startswith('CLASSIFICATION:'):
                type_str = line.replace('CLASSIFICATION:', '').strip()
                for char in type_str:
                    if char.upper() in 'OCLN':
                        try:
                            bond_type = BondType.from_string(char)
                        except:
                            pass
                        break
            elif line.startswith('CONFIDENCE:'):
                try:
                    confidence = float(line.replace('CONFIDENCE:', '').strip())
                    confidence = max(0.0, min(1.0, confidence))
                except:
                    pass
            elif line.startswith('REASONING:'):
                reasoning = line.replace('REASONING:', '').strip()
        
        return EvaluationResult(
            scenario_id=scenario_id,
            condition=condition,
            bond_type=bond_type,
            confidence=confidence,
            reasoning=reasoning,
            raw_response=response,
            model=self.model
        )


class SimulatedEvaluator(BaseEvaluator):
    """Simulates responses using NA-SQND theory."""
    
    def __init__(self):
        super().__init__("simulated-na-sqnd")
        self.theta_by_condition = {
            'control': 0.0,
            'weak': np.pi / 6,
            'strong': np.pi / 3
        }
    
    def evaluate(self, scenario: AITAScenario, condition: ThresholdCondition) -> EvaluationResult:
        self.request_count += 1
        
        # Get theoretical theta with variation
        theta = self.theta_by_condition.get(condition.name, 0.0)
        verdict_adj = {'NTA': -0.1, 'YTA': 0.2, 'ESH': 0.1, 'NAH': 0.0}
        theta += verdict_adj.get(scenario.label, 0.0)
        theta += np.random.normal(0, 0.15)
        theta = max(0, min(np.pi, theta))
        
        # Sample from POVM
        eta = 0.75
        p_O_slot = np.cos(theta / 2) ** 2
        p_C_slot = np.sin(theta / 2) ** 2
        
        probs = {
            BondType.OBLIGATION: (1 + eta) / 2 * p_O_slot,
            BondType.LIBERTY: (1 - eta) / 2 * p_O_slot,
            BondType.CLAIM: (1 + eta) / 2 * p_C_slot,
            BondType.NO_CLAIM: (1 - eta) / 2 * p_C_slot,
        }
        
        bond_types = list(probs.keys())
        prob_values = np.array([probs[bt] for bt in bond_types])
        prob_values /= prob_values.sum()
        
        selected = np.random.choice(bond_types, p=prob_values)
        
        return EvaluationResult(
            scenario_id=scenario.id,
            condition=condition.name,
            bond_type=selected,
            confidence=0.7 + np.random.uniform(0, 0.25),
            reasoning="[Simulated response]",
            raw_response="[SIMULATED]",
            model=self.model
        )


class OllamaEvaluator(BaseEvaluator):
    """Uses Ollama for local model inference."""
    
    def __init__(self, model: str = "llama3.2", base_url: str = "http://localhost:11434"):
        super().__init__(model)
        self.base_url = base_url
        self.api_url = f"{base_url}/api/generate"
    
    def evaluate(self, scenario: AITAScenario, condition: ThresholdCondition) -> EvaluationResult:
        self.request_count += 1
        prompt = self.create_prompt(scenario, condition)
        
        try:
            response = requests.post(
                self.api_url,
                json={
                    "model": self.model,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": 0.7,
                        "num_predict": 200
                    }
                },
                timeout=60
            )
            response.raise_for_status()
            result = response.json()
            raw_text = result.get("response", "")
            return self.parse_response(raw_text, scenario.id, condition.name)
            
        except Exception as e:
            print(f"  ⚠ Ollama error: {e}")
            return EvaluationResult(
                scenario_id=scenario.id,
                condition=condition.name,
                bond_type=BondType.LIBERTY,
                confidence=0.0,
                reasoning=f"Error: {e}",
                raw_response="",
                model=self.model
            )


class AnthropicEvaluator(BaseEvaluator):
    """Uses Anthropic Claude API."""
    
    def __init__(self, api_key: str, model: str = "claude-sonnet-4-20250514"):
        super().__init__(model)
        import anthropic
        self.client = anthropic.Anthropic(api_key=api_key)
    
    def evaluate(self, scenario: AITAScenario, condition: ThresholdCondition) -> EvaluationResult:
        self.request_count += 1
        prompt = self.create_prompt(scenario, condition)
        
        try:
            response = self.client.messages.create(
                model=self.model,
                max_tokens=300,
                messages=[{"role": "user", "content": prompt}]
            )
            raw_text = response.content[0].text
            return self.parse_response(raw_text, scenario.id, condition.name)
            
        except Exception as e:
            print(f"  ⚠ Anthropic error: {e}")
            return EvaluationResult(
                scenario_id=scenario.id,
                condition=condition.name,
                bond_type=BondType.LIBERTY,
                confidence=0.0,
                reasoning=f"Error: {e}",
                raw_response="",
                model=self.model
            )


def get_evaluator(backend: str) -> BaseEvaluator:
    """Factory function to create the appropriate evaluator."""
    if backend == "simulation":
        return SimulatedEvaluator()
    elif backend == "ollama":
        return OllamaEvaluator(model=OLLAMA_MODEL, base_url=OLLAMA_URL)
    elif backend == "anthropic":
        if not ANTHROPIC_API_KEY:
            raise ValueError("ANTHROPIC_API_KEY is required for anthropic backend")
        return AnthropicEvaluator(api_key=ANTHROPIC_API_KEY, model=ANTHROPIC_MODEL)
    else:
        raise ValueError(f"Unknown backend: {backend}")

print("✓ Evaluator classes defined!")

## 6. Run the Experiment

In [ ]:
def run_experiment(evaluator: BaseEvaluator, scenarios: List[AITAScenario], 
                   conditions: List[str]) -> List[EvaluationResult]:
    """Run the full experiment."""
    results = []
    total = len(scenarios) * len(conditions)
    
    print("\n🧪 Running Protocol 1 Experiment")
    print(f"   Backend: {evaluator.model}")
    print(f"   Scenarios: {len(scenarios)}")
    print(f"   Conditions: {conditions}")
    print(f"   Total evaluations: {total}\n")
    
    for i, scenario in enumerate(scenarios):
        for j, cond_name in enumerate(conditions):
            condition = THRESHOLD_CONDITIONS[cond_name]
            idx = i * len(conditions) + j + 1
            
            print(f"  [{idx}/{total}] {scenario.id[:12]:12s} | {cond_name:8s}", end=" ")
            
            result = evaluator.evaluate(scenario, condition)
            results.append(result)
            
            print(f"→ {result.bond_type.value} (conf: {result.confidence:.2f})")
            
            # Small delay for API backends
            if not isinstance(evaluator, SimulatedEvaluator):
                time.sleep(0.5)
    
    print(f"\n✓ Completed {len(results)} evaluations")
    return results


# Run the experiment
evaluator = get_evaluator(BACKEND)
scenarios_to_use = SCENARIOS[:N_SCENARIOS]
results = run_experiment(evaluator, scenarios_to_use, CONDITIONS)

## 7. Analysis

In [ ]:
def estimate_theta(responses: List[BondType]) -> Tuple[float, float]:
    """Estimate mixing angle θ from responses."""
    counts = {bt: 0 for bt in BondType}
    for r in responses:
        counts[r] += 1
    
    n = len(responses)
    if n == 0:
        return 0.0, np.inf
    
    p_O = counts[BondType.OBLIGATION] / n
    p_L = counts[BondType.LIBERTY] / n
    p_C = counts[BondType.CLAIM] / n
    p_N = counts[BondType.NO_CLAIM] / n
    
    denom = p_O + p_L
    numer = p_C + p_N
    
    if denom < 1e-10:
        theta = np.pi
    elif numer < 1e-10:
        theta = 0.0
    else:
        theta = 2 * np.arctan(np.sqrt(numer / denom))
    
    # Bootstrap SE
    if n >= 5:
        theta_samples = []
        for _ in range(500):
            sample = random.choices(responses, k=n)
            counts_s = {bt: sum(1 for r in sample if r == bt) for bt in BondType}
            p_O_s = counts_s[BondType.OBLIGATION] / n
            p_L_s = counts_s[BondType.LIBERTY] / n
            p_C_s = counts_s[BondType.CLAIM] / n
            p_N_s = counts_s[BondType.NO_CLAIM] / n
            denom_s = p_O_s + p_L_s
            numer_s = p_C_s + p_N_s
            if denom_s < 1e-10:
                theta_samples.append(np.pi)
            elif numer_s < 1e-10:
                theta_samples.append(0.0)
            else:
                theta_samples.append(2 * np.arctan(np.sqrt(numer_s / denom_s)))
        se = np.std(theta_samples)
    else:
        se = np.inf
    
    return theta, se


def analyze_results(results: List[EvaluationResult]) -> Dict:
    """Analyze experiment results."""
    analysis = {'conditions': {}}
    
    for cond in CONDITIONS:
        cond_results = [r for r in results if r.condition == cond]
        responses = [r.bond_type for r in cond_results]
        
        theta, se = estimate_theta(responses)
        
        counts = {bt.value: sum(1 for r in responses if r == bt) for bt in BondType}
        
        analysis['conditions'][cond] = {
            'n': len(cond_results),
            'theta': theta,
            'theta_se': se,
            'theta_degrees': np.degrees(theta),
            'distribution': counts,
            'mean_confidence': np.mean([r.confidence for r in cond_results])
        }
    
    return analysis


# Run analysis
analysis = analyze_results(results)

# Print summary
print("\n" + "="*60)
print("PROTOCOL 1 RESULTS: BOND-TYPE ROTATION")
print("="*60)

print("\n📊 Results by Condition:\n")
print(f"{'Condition':<12} {'θ (rad)':<10} {'θ (deg)':<10} {'Distribution'}")
print("-"*60)

for cond in CONDITIONS:
    data = analysis['conditions'][cond]
    dist_str = ", ".join(f"{k}={v}" for k,v in data['distribution'].items() if v > 0)
    print(f"{cond:<12} {data['theta']:<10.4f} {data['theta_degrees']:<10.1f} {dist_str}")

# Check theoretical predictions
print("\n📐 Comparison to NA-SQND Predictions:\n")
theoretical = {'control': 0, 'weak': np.pi/6, 'strong': np.pi/3}
print(f"{'Condition':<12} {'Predicted':<12} {'Observed':<12} {'Δ'}")
print("-"*50)
for cond in CONDITIONS:
    pred = np.degrees(theoretical[cond])
    obs = analysis['conditions'][cond]['theta_degrees']
    delta = obs - pred
    print(f"{cond:<12} {pred:<12.1f}° {obs:<12.1f}° {delta:+.1f}°")

## 8. Visualization

In [ ]:
# Figure 1: Theta by Condition
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Bar chart of theta
ax1 = axes[0]
conditions = list(analysis['conditions'].keys())
thetas = [analysis['conditions'][c]['theta'] for c in conditions]
theta_ses = [analysis['conditions'][c]['theta_se'] for c in conditions]

colors = ['#2ecc71', '#f39c12', '#e74c3c']
bars = ax1.bar(conditions, thetas, yerr=theta_ses, capsize=5, color=colors, 
               edgecolor='black', alpha=0.7)

# Add theoretical predictions
theoretical_vals = [0, np.pi/6, np.pi/3]
ax1.scatter(conditions, theoretical_vals, marker='*', s=200, color='black', 
            zorder=5, label='NA-SQND Prediction')

ax1.set_ylabel('θ (Mixing Angle, radians)', fontsize=12)
ax1.set_xlabel('Condition', fontsize=12)
ax1.set_title('Bond-Type Rotation by Threshold Condition', fontsize=14)
ax1.legend()

# Add degree labels on right y-axis
ax1_deg = ax1.twinx()
ax1_deg.set_ylim([np.degrees(y) for y in ax1.get_ylim()])
ax1_deg.set_ylabel('θ (degrees)', fontsize=12)

# Plot 2: Response distributions
ax2 = axes[1]
x = np.arange(len(conditions))
width = 0.2

bond_colors = {'O': '#3498db', 'C': '#e74c3c', 'L': '#2ecc71', 'N': '#9b59b6'}

for i, bt in enumerate(['O', 'C', 'L', 'N']):
    values = [analysis['conditions'][c]['distribution'].get(bt, 0) / 
              analysis['conditions'][c]['n'] for c in conditions]
    ax2.bar(x + i*width, values, width, label=bt, color=bond_colors[bt], 
            edgecolor='black', alpha=0.7)

ax2.set_xticks(x + 1.5*width)
ax2.set_xticklabels(conditions)
ax2.set_ylabel('Proportion', fontsize=12)
ax2.set_xlabel('Condition', fontsize=12)
ax2.set_title('Response Distribution by Condition', fontsize=14)
ax2.legend(title='Bond Type')
ax2.set_ylim(0, 1)

plt.tight_layout()
plt.savefig('protocol1_results.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Figure saved to protocol1_results.png")

In [ ]:
# Figure 2: Detailed response breakdown
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, cond in zip(axes, CONDITIONS):
    dist = analysis['conditions'][cond]['distribution']
    n = analysis['conditions'][cond]['n']
    
    labels = ['O\n(Obligation)', 'C\n(Claim)', 'L\n(Liberty)', 'N\n(No-claim)']
    values = [dist.get('O', 0), dist.get('C', 0), dist.get('L', 0), dist.get('N', 0)]
    colors_pie = ['#3498db', '#e74c3c', '#2ecc71', '#9b59b6']
    
    # Only show non-zero slices
    labels_nz = [l for l, v in zip(labels, values) if v > 0]
    values_nz = [v for v in values if v > 0]
    colors_nz = [c for c, v in zip(colors_pie, values) if v > 0]
    
    if values_nz:
        wedges, texts, autotexts = ax.pie(values_nz, labels=labels_nz, colors=colors_nz,
                                          autopct='%1.0f%%', startangle=90)
        for autotext in autotexts:
            autotext.set_fontsize(10)
            autotext.set_fontweight('bold')
    
    theta_deg = analysis['conditions'][cond]['theta_degrees']
    ax.set_title(f"{cond.capitalize()}\nθ = {theta_deg:.1f}°", fontsize=12)

plt.suptitle('Response Distributions by Condition', fontsize=14, y=1.02)
plt.tight_layout()
plt.savefig('protocol1_distributions.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Statistical Tests & Falsification Assessment

In [ ]:
print("\n" + "="*60)
print("STATISTICAL TESTS & FALSIFICATION ASSESSMENT")
print("="*60)

# Chi-square tests for distribution differences
print("\n📊 Chi-Square Tests for Distribution Differences:\n")

for cond in ['weak', 'strong']:
    control_dist = analysis['conditions']['control']['distribution']
    cond_dist = analysis['conditions'][cond]['distribution']
    
    # Create contingency table
    all_types = ['O', 'C', 'L', 'N']
    control_counts = [control_dist.get(t, 0) for t in all_types]
    cond_counts = [cond_dist.get(t, 0) for t in all_types]
    
    # Only test if we have non-zero counts
    if sum(control_counts) > 0 and sum(cond_counts) > 0:
        contingency = np.array([control_counts, cond_counts])
        # Remove zero columns
        contingency = contingency[:, contingency.sum(axis=0) > 0]
        
        if contingency.shape[1] > 1:
            chi2, p, dof, expected = stats.chi2_contingency(contingency)
            print(f"  Control vs {cond}: χ²={chi2:.2f}, df={dof}, p={p:.4f}")
            print(f"    → {'Significant' if p < 0.05 else 'Not significant'} at α=0.05")
        else:
            print(f"  Control vs {cond}: Insufficient variation for test")

# Falsification criteria
print("\n🔬 Falsification Criteria:\n")

thetas = {c: analysis['conditions'][c]['theta'] for c in CONDITIONS}

# Criterion 1: No threshold effect
theta_range = max(thetas.values()) - min(thetas.values())
no_effect = theta_range < 0.1
print("  1. No threshold effect (θ range < 0.1):")
print(f"     θ range = {theta_range:.4f} → {'FALSIFIED ❌' if no_effect else 'NOT FALSIFIED ✓'}")

# Criterion 2: No graded response
graded = thetas['control'] <= thetas['weak'] <= thetas['strong']
print("  2. No graded response (control ≤ weak ≤ strong):")
print(f"     {thetas['control']:.3f} ≤ {thetas['weak']:.3f} ≤ {thetas['strong']:.3f}")
print(f"     → {'NOT FALSIFIED ✓' if graded else 'FALSIFIED ❌'}")

# Criterion 3: Wrong direction
wrong_dir = thetas['strong'] < thetas['control']
print("  3. Wrong direction (strong < control):")
print(f"     → {'FALSIFIED ❌' if wrong_dir else 'NOT FALSIFIED ✓'}")

# Overall assessment
theory_falsified = no_effect or not graded or wrong_dir
print("\n" + "="*60)
if theory_falsified:
    print("❌ OVERALL: Theory FALSIFIED by this data")
else:
    print("✅ OVERALL: Theory NOT FALSIFIED - Results consistent with NA-SQND")
print("="*60)

## 10. Export Results

In [ ]:
# Create results DataFrame
df_results = pd.DataFrame([
    {
        'scenario_id': r.scenario_id,
        'condition': r.condition,
        'bond_type': r.bond_type.value,
        'confidence': r.confidence,
        'reasoning': r.reasoning[:100],
        'model': r.model
    }
    for r in results
])

# Save to CSV
df_results.to_csv('protocol1_results.csv', index=False)
print("✓ Results saved to protocol1_results.csv")

# Display sample
print("\n📋 Sample Results:")
display(df_results.head(10))

In [ ]:
# Create summary DataFrame
df_summary = pd.DataFrame([
    {
        'Condition': cond,
        'N': data['n'],
        'θ (rad)': data['theta'],
        'θ (deg)': data['theta_degrees'],
        'SE': data['theta_se'],
        'Mean Confidence': data['mean_confidence'],
        'O': data['distribution'].get('O', 0),
        'C': data['distribution'].get('C', 0),
        'L': data['distribution'].get('L', 0),
        'N': data['distribution'].get('N', 0),
    }
    for cond, data in analysis['conditions'].items()
])

df_summary.to_csv('protocol1_summary.csv', index=False)
print("✓ Summary saved to protocol1_summary.csv")

print("\n📊 Summary Table:")
display(df_summary)

In [ ]:
# Download files (Colab)
try:
    from google.colab import files
    files.download('protocol1_results.csv')
    files.download('protocol1_summary.csv')
    files.download('protocol1_results.png')
    files.download('protocol1_distributions.png')
    print("✓ Files downloaded!")
except:
    print("ℹ️ Not running on Colab - files saved locally")

---

## 📖 Interpretation Guide

### What do the results mean?

**θ (Mixing Angle)**: Measures the "rotation" in moral judgment space
- θ ≈ 0°: Responses cluster in O/L (poster has obligations or is free)
- θ ≈ 90°: Responses cluster in C/N (other party has claims or not)
- θ ≈ 45°: Mixed responses

**NA-SQND Predictions**:
- **Control**: θ ≈ 0° (no threshold crossed)
- **Weak threshold**: θ ≈ 30° (minor boundary crossing)
- **Strong threshold**: θ ≈ 60° (major boundary crossing)

**Falsification**:
- If θ doesn't increase with threshold strength, theory is falsified
- If responses don't show systematic rotation, theory is falsified

### Next Steps
1. Run with more scenarios (N=150) for stronger statistical power
2. Try different LLM models to test consistency
3. Compare simulation to real model responses
4. Implement Protocol 2 (holonomy path dependence)

---

*Based on: Bond, A.H. (2026). Non-Abelian Gauge Structure in Stratified Quantum Normative Dynamics, v3.4*